<a href="https://colab.research.google.com/github/sidd8rth/A-Z-ML-Udemy/blob/main/duplicate_questions_sentenceBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fuzzywuzzy
!pip install hyperopt
!pip install sentence_transformers
!pip install imbalanced-learn
import nltk
nltk.download('omw-1.4')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 5.2 MB/s 
     |████████████████████████████████| 4.4 MB 61.4 MB/s 
     |████████████████████████████████| 1.2 MB 57.3 MB/s 
     |████████████████████████████████| 101 kB 12.3 MB/s 
     |████████████████████████████████| 596 kB 66.6 MB/s 
     |████████████████████████████████| 6.6 MB 62.0 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=95b376aed6d09193594f4dfdd3429f0c495999de85572a8ab8584dac44c71011
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers
  Attempting uninstall: pyya

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
import re
from fuzzywuzzy import fuzz
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, log_loss
from sklearn.calibration import CalibratedClassifierCV
import xgboost as xgb
import nltk
import time
from matplotlib.pyplot import figure
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import MinMaxScaler
from joblib import dump, load
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.model_selection import ShuffleSplit
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import hyperopt
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import warnings
import gc
from sklearn.model_selection import cross_val_score
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

/usr/local/lib/python3.7/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
data = pd.read_csv('/content/drive/MyDrive/Model/train.csv')

In [6]:
data = data.loc[:,:]

In [7]:
data.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1001 non-null   int64 
 1   qid1          1001 non-null   int64 
 2   qid2          1001 non-null   int64 
 3   question1     1001 non-null   object
 4   question2     1001 non-null   object
 5   is_duplicate  1001 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


In [9]:
data = data.dropna()
print(data.shape)

(1001, 6)


In [10]:
print(data.duplicated(('question1', 'question2')).sum())

0


# Data Cleaning

In [11]:
def preprocess_text(x):
    x = str(x).lower()
    x = x.replace(",000,000", "m").replace(",000", "k").replace("′", "'").replace("’", "'")\
                           .replace("won't", "will not").replace("cannot", "can not").replace("can't", "can not")\
                           .replace("n't", " not").replace("what's", "what is").replace("it's", "it is")\
                           .replace("'ve", " have").replace("i'm", "i am").replace("'re", " are")\
                           .replace("he's", "he is").replace("she's", "she is").replace("'s", " own")\
                           .replace("%", " percent ").replace("₹", " rupee ").replace("$", " dollar ")\
                           .replace("€", " euro ").replace("'ll", " will")
    x = re.sub(r"([0-9]+)000000", r"\1m", x)
    x = re.sub(r"([0-9]+)000", r"\1k", x)
    x = re.sub(r"http\S+", "", x)
    x = re.sub('\W', ' ', x)
    
    lemmatizer = WordNetLemmatizer()
    x = lemmatizer.lemmatize(x)
    bfs = BeautifulSoup(x)
    x = bfs.get_text()
    x = x.strip()
    return x

In [12]:
def data_cleaning(data):
    newdata = pd.DataFrame()
    newdata['question1_final'] = data.question1.apply(preprocess_text)
    newdata['question2_final'] = data.question2.apply(preprocess_text)
    return newdata

In [13]:
traindata = data_cleaning(data)

In [14]:
print(data.head())

   id  qid1  qid2                                          question1  \
0   0     1     2  What is the step by step guide to invest in sh...   
1   1     3     4  What is the story of Kohinoor (Koh-i-Noor) Dia...   
2   2     5     6  How can I increase the speed of my internet co...   
3   3     7     8  Why am I mentally very lonely? How can I solve...   
4   4     9    10  Which one dissolve in water quikly sugar, salt...   

                                           question2  is_duplicate  
0  What is the step by step guide to invest in sh...             0  
1  What would happen if the Indian government sto...             0  
2  How can Internet speed be increased by hacking...             0  
3  Find the remainder when [math]23^{24}[/math] i...             0  
4            Which fish would survive in salt water?             0  


In [15]:
print(traindata.head())

                                     question1_final  \
0  what is the step by step guide to invest in sh...   
1  what is the story of kohinoor  koh i noor  dia...   
2  how can i increase the speed of my internet co...   
3  why am i mentally very lonely  how can i solve it   
4  which one dissolve in water quikly sugar  salt...   

                                     question2_final  
0  what is the step by step guide to invest in sh...  
1  what would happen if the indian government sto...  
2  how can internet speed be increased by hacking...  
3  find the remainder when  math 23  24   math  i...  
4             which fish would survive in salt water  


# Feature Extraction

In [16]:
def doesMatch (q, match):
    q1, q2 = q['question1_final'], q['question2_final']
    q1 = q1.split()
    q2 = q2.split()
    if len(q1)>0 and len(q2)>0 and q1[match]==q2[match]:
        return 1
    else:
        return 0

In [17]:
def feature_extract(data):
    data['q1_char_num'] = data.question1_final.apply(len)
    data['q2_char_num'] = data.question2_final.apply(len)
    data['q1_word_num'] = data.question1_final.apply(lambda x: len(x.split()))
    data['q2_word_num'] = data.question2_final.apply(lambda x: len(x.split()))
    
    data['total_word_num'] = data['q1_word_num'] + data['q2_word_num']
    data['differ_word_num'] = abs(data['q1_word_num'] - data['q2_word_num'])
    data['same_first_word'] = data.apply(lambda x: doesMatch(x, 0) ,axis=1)
    data['same_last_word'] = data.apply(lambda x: doesMatch(x, -1) ,axis=1)
    data['total_unique_word_num'] = data.apply(lambda x: len(set(x.question1_final.split()).union(set(x.question2_final.split()))) ,axis=1)
    data['total_unique_word_withoutstopword_num'] = data.apply(lambda x: len(set(x.question1_final.split()).union(set(x.question2_final.split())) - set(stopwords.words('english'))) ,axis=1)
    data['total_unique_word_num_ratio'] = data['total_unique_word_num'] / data['total_word_num']
    
    data['common_word_num'] = data.apply(lambda x: len(set(x.question1_final.split()).intersection(set(x.question2_final.split()))) ,axis=1)
    data['common_word_ratio'] = data['common_word_num'] / data['total_unique_word_num']
    data['common_word_ratio_min'] = data['common_word_num'] / data.apply(lambda x: min(len(set(x.question1_final.split())), len(set(x.question2_final.split()))) ,axis=1) 
    data['common_word_ratio_max'] = data['common_word_num'] / data.apply(lambda x: max(len(set(x.question1_final.split())), len(set(x.question2_final.split()))) ,axis=1) 
    
    data['common_word_withoutstopword_num'] = data.apply(lambda x: len(set(x.question1_final.split()).intersection(set(x.question2_final.split())) - set(stopwords.words('english'))) ,axis=1)
    data['common_word_withoutstopword_ratio'] = data['common_word_withoutstopword_num'] / data['total_unique_word_withoutstopword_num']
    data['common_word_withoutstopword_ratio_min'] = data['common_word_withoutstopword_num'] / data.apply(lambda x: min(len(set(x.question1_final.split()) - set(stopwords.words('english'))), len(set(x.question2_final.split()) - set(stopwords.words('english')))) ,axis=1) 
    data['common_word_withoutstopword_ratio_max'] = data['common_word_withoutstopword_num'] / data.apply(lambda x: max(len(set(x.question1_final.split()) - set(stopwords.words('english'))), len(set(x.question2_final.split()) - set(stopwords.words('english')))) ,axis=1) 
    
    data["fuzz_ratio"] = data.apply(lambda x: fuzz.ratio(x.question1_final, x.question2_final), axis=1)
    data["fuzz_partial_ratio"] = data.apply(lambda x: fuzz.partial_ratio(x.question1_final, x.question2_final), axis=1)
    data["fuzz_token_set_ratio"] = data.apply(lambda x: fuzz.token_set_ratio(x.question1_final, x.question2_final), axis=1)
    data["fuzz_token_sort_ratio"] = data.apply(lambda x: fuzz.token_sort_ratio(x.question1_final, x.question2_final), axis=1)
    data.fillna(0, inplace=True)
    return data

In [18]:
traindata = feature_extract(traindata)

In [19]:
traindata.head()

,question1_final,question2_final,q1_char_num,q2_char_num,q1_word_num,q2_word_num,total_word_num,differ_word_num,same_first_word,same_last_word,...,common_word_ratio_min,common_word_ratio_max,common_word_withoutstopword_num,common_word_withoutstopword_ratio,common_word_withoutstopword_ratio_min,common_word_withoutstopword_ratio_max,fuzz_ratio,fuzz_partial_ratio,fuzz_token_set_ratio,fuzz_token_sort_ratio
0,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,65,56,14,12,26,2,1,0,...,1.000000,0.916667,5,0.833333,1.0,0.833333,93,100,100,93
1,what is the story of kohinoor koh i noor dia...,what would happen if the indian government sto...,50,87,10,15,25,5,1,0,...,0.700000,0.500000,4,0.363636,0.8,0.400000,66,74,86,63
2,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...,72,58,14,10,24,4,1,0,...,0.400000,0.285714,2,0.222222,0.4,0.333333,43,46,63,63
3,why am i mentally very lonely how can i solve it,find the remainder when math 23 24 math i...,49,64,11,13,24,2,0,0,...,0.000000,0.000000,0,0.000000,0.0,0.000000,9,11,28,24
4,which one dissolve in water quikly sugar salt...,which fish would survive in salt water,75,38,13,7,20,6,1,0,...,0.571429,0.307692,2,0.153846,0.4,0.200000,35,55,67,47


In [20]:
traindata.shape

(1001, 25)

# Featurization with SentenceBERT

In [21]:
modelST = SentenceTransformer('paraphrase-mpnet-base-v2')

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.70k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/594 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [22]:
def getBertEmbeddings(data, filename):
    batch = 5
    with open(filename, 'wb') as f:
        while(len(data)):
            tempdata = data[:batch]
            data = data[batch:]
            tempembed = modelST.encode(tempdata.values, device='cuda')
            np.save(f, tempembed, allow_pickle=True)

In [23]:
getBertEmbeddings(traindata.question1_final, '/content/drive/MyDrive/Model/temp_train_question1_sentenceBERT.npy')
getBertEmbeddings(traindata.question2_final, '/content/drive/MyDrive/Model/temp_train_question2_sentenceBERT.npy')

In [24]:
def cosine_euclidean(u, v):
    return np.array([np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v)), np.linalg.norm(u - v)])

In [25]:
with open('/content/drive/MyDrive/Model/temp_train_question1_sentenceBERT.npy', 'rb') as q1_vec, open('/content/drive/MyDrive/Model/temp_train_question2_sentenceBERT.npy', 'rb') as q2_vec:
    distances = []
    while True:
        try:
            q1_20k = np.load(q1_vec, allow_pickle=True)
            q2_20k = np.load(q2_vec, allow_pickle=True)
            for q1,q2 in zip(q1_20k, q2_20k):
                dists = cosine_euclidean(q1, q2)
                distances.append(dists)
        except IOError as e:
            distances = np.array(distances)
            break
distances = pd.DataFrame(distances, columns=['cosine_simlarity_bert', 'euclidean_distance_bert'])
traindata = pd.concat([traindata, pd.DataFrame(distances)], axis=1)

In [26]:
data.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [27]:
traindata.head()

,question1_final,question2_final,q1_char_num,q2_char_num,q1_word_num,q2_word_num,total_word_num,differ_word_num,same_first_word,same_last_word,...,common_word_withoutstopword_num,common_word_withoutstopword_ratio,common_word_withoutstopword_ratio_min,common_word_withoutstopword_ratio_max,fuzz_ratio,fuzz_partial_ratio,fuzz_token_set_ratio,fuzz_token_sort_ratio,cosine_simlarity_bert,euclidean_distance_bert
0,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,65,56,14,12,26,2,1,0,...,5,0.833333,1.0,0.833333,93,100,100,93,0.891494,1.531087
1,what is the story of kohinoor koh i noor dia...,what would happen if the indian government sto...,50,87,10,15,25,5,1,0,...,4,0.363636,0.8,0.400000,66,74,86,63,0.603554,2.819399
2,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...,72,58,14,10,24,4,1,0,...,2,0.222222,0.4,0.333333,43,46,63,63,0.654107,2.714901
3,why am i mentally very lonely how can i solve it,find the remainder when math 23 24 math i...,49,64,11,13,24,2,0,0,...,0,0.000000,0.0,0.000000,9,11,28,24,0.154826,4.354044
4,which one dissolve in water quikly sugar salt...,which fish would survive in salt water,75,38,13,7,20,6,1,0,...,2,0.153846,0.4,0.200000,35,55,67,47,0.262982,3.852257


In [28]:
traindata.shape

(1001, 27)

# Data Preprocess

In [29]:
traindata.drop(columns=['question1_final', 'question2_final'], inplace=True)
traindata = traindata.to_numpy()
scaler = MinMaxScaler()
scaler.fit(traindata)
traindata = scaler.transform(traindata)

In [30]:
def loadVectors(filename):
    with open(filename, 'rb') as f:
        q_vectors = []
        while True:
            try:
                q_vec = np.load(f, allow_pickle=True)
                q_vectors.extend(list(q_vec))
            except IOError as e:
                q_vectors = np.array(q_vectors)
                break
    return q_vectors

In [31]:
train_question1_vec = loadVectors('/content/drive/MyDrive/Model/temp_train_question1_sentenceBERT.npy')

In [32]:
train_question2_vec = loadVectors('/content/drive/MyDrive/Model/temp_train_question2_sentenceBERT.npy')

In [33]:
traindata.shape

(1001, 25)

In [34]:
train_question1_vec.shape

(1001, 768)

In [35]:
train_question2_vec.shape

(1001, 768)

In [36]:
traindata = np.hstack((traindata, train_question1_vec, train_question2_vec))

In [37]:
traindata.shape

(1001, 1561)

In [38]:
traindata

array([[ 0.23404255,  0.215311  ,  0.31428571, ...,  0.09340317,
        -0.00154406,  0.00262191],
       [ 0.17021277,  0.36363636,  0.2       , ...,  0.00670765,
        -0.15125999,  0.0771141 ],
       [ 0.26382979,  0.22488038,  0.31428571, ...,  0.03992499,
        -0.09637078, -0.08439203],
       ...,
       [ 0.57021277,  0.59808612,  0.6       , ...,  0.04378054,
        -0.05996016,  0.07026204],
       [ 0.10638298,  0.0861244 ,  0.14285714, ...,  0.12254233,
         0.05001653, -0.01822244],
       [ 0.20425532,  0.11961722,  0.28571429, ..., -0.10861464,
        -0.02964454, -0.07883472]])

In [39]:
oversample = RandomOverSampler(sampling_strategy='minority')
X_train, y_train = oversample.fit_resample(traindata, data.is_duplicate.to_numpy())

In [40]:
print(np.count_nonzero(y_train == 0))
print(np.count_nonzero(y_train == 1))

621
621


# Model Track

In [41]:
splits = ShuffleSplit(n_splits=1, test_size=.3, random_state=42)

In [42]:
rf_param_grid = {
                    'n_estimators':[200, 500, 800], 
                    'min_samples_split':[5, 15],
                    'max_depth': [70, 150, None]
                }

In [43]:
rf_clf = RandomForestClassifier()

In [44]:
rf_clf_search = HalvingGridSearchCV(rf_clf, rf_param_grid, cv=splits, factor=2, scoring='accuracy', verbose=3)

In [45]:
rf_clf_search.fit(X_train, y_train)

n_iterations: 5
n_required_iterations: 5
n_possible_iterations: 5
min_resources_: 77
max_resources_: 1242
aggressive_elimination: False
factor: 2
----------
iter: 0
n_candidates: 18
n_resources: 77
Fitting 1 folds for each of 18 candidates, totalling 18 fits
[CV 1/1] END max_depth=70, min_samples_split=5, n_estimators=200;, score=(train=1.000, test=0.652) total time=   0.3s
[CV 1/1] END max_depth=70, min_samples_split=5, n_estimators=500;, score=(train=1.000, test=0.565) total time=   0.7s
[CV 1/1] END max_depth=70, min_samples_split=5, n_estimators=800;, score=(train=1.000, test=0.565) total time=   1.2s
[CV 1/1] END max_depth=70, min_samples_split=15, n_estimators=200;, score=(train=1.000, test=0.565) total time=   0.3s
[CV 1/1] END max_depth=70, min_samples_split=15, n_estimators=500;, score=(train=1.000, test=0.609) total time=   0.7s
[CV 1/1] END max_depth=70, min_samples_split=15, n_estimators=800;, score=(train=1.000, test=0.652) total time=   1.1s
[CV 1/1] END max_depth=150, mi

HalvingGridSearchCV(cv=ShuffleSplit(n_splits=1, random_state=42, test_size=0.3, train_size=None),
                    estimator=RandomForestClassifier(), factor=2,
                    param_grid={'max_depth': [70, 150, None],
                                'min_samples_split': [5, 15],
                                'n_estimators': [200, 500, 800]},
                    scoring='accuracy', verbose=3)

In [46]:
rf_clf_search.best_params_

{'max_depth': 70, 'min_samples_split': 15, 'n_estimators': 500}

In [47]:
rf_clf_search.best_score_

0.8672086720867209

In [48]:
rf_clf_model = rf_clf_search.best_estimator_

In [49]:
rf_clf_model

RandomForestClassifier(max_depth=70, min_samples_split=15, n_estimators=500)

# Proposed Testing Code

In [50]:
def getBertEmbeddings(data, filename):
  with open(filename, 'wb') as f:
    tempdata = data
    tempembed = modelST.encode(tempdata.values, device='cuda')
    np.save(f, tempembed, allow_pickle=True)

def predict(x1,x2):
  # data cleaning
  data = [[x1,x2]]
  df = pd.DataFrame(data, columns=['question1','question2'])
  #print(df.head())
  newdata = data_cleaning(df)
  # Feature Extraction
  newdata = feature_extract(newdata)
  # sentenceBERT
  getBertEmbeddings(newdata.question1_final, '/content/drive/MyDrive/Model/temp_new_question1_sentenceBERT.npy')
  getBertEmbeddings(newdata.question2_final, '/content/drive/MyDrive/Model/temp_new_question2_sentenceBERT.npy')


  #q1_test = np.array()
  with open('/content/drive/MyDrive/Model/temp_new_question1_sentenceBERT.npy', 'rb') as q1_vec, open('/content/drive/MyDrive/Model/temp_new_question2_sentenceBERT.npy', 'rb') as q2_vec:
    distances = []
    while True:
        try:
            q1_20k = np.load(q1_vec, allow_pickle=True)
            q2_20k = np.load(q2_vec, allow_pickle=True)
            for q1,q2 in zip(q1_20k, q2_20k):
                dists = cosine_euclidean(q1, q2)
                distances.append(dists)
        except IOError as e:
            distances = np.array(distances)
            break
  distances = pd.DataFrame(distances, columns=['cosine_simlarity_bert', 'euclidean_distance_bert'])
  newdata = pd.concat([newdata, pd.DataFrame(distances)], axis=1)


  #pre processing
  newdata.drop(columns=['question1_final', 'question2_final'], inplace=True)
  newdata = newdata.to_numpy()
  newdata = scaler.transform(newdata)


  with open('/content/drive/MyDrive/Model/temp_newdata.npy', 'wb') as f:
    tempdata = newdata
    np.save(f, tempdata, allow_pickle=True)


  # testing
  with open('/content/drive/MyDrive/Model/temp_newdata.npy', 'rb') as X_test_1, \
  open('/content/drive/MyDrive/Model/temp_new_question1_sentenceBERT.npy', 'rb') as X_test_q1, \
  open('/content/drive/MyDrive/Model/temp_new_question2_sentenceBERT.npy', 'rb') as X_test_q2:
    y_pred_proba_rf = []
    while True:
      try:
          test_20k = np.load(X_test_1, allow_pickle=True)
          q1_20k = np.load(X_test_q1, allow_pickle=True)
          q2_20k = np.load(X_test_q2, allow_pickle=True)
          print(test_20k.shape)
          print(q1_20k.shape)
          print(q2_20k.shape)
          X_test = np.hstack((test_20k, q1_20k, q2_20k))
          #print(X_test)
          y_pred_proba_rf.extend(list(rf_clf_model.predict(X_test)))
      except IOError as e:
          break
    return y_pred_proba_rf[0]

q1 = "What are advantages of python?"
q2 = "How are you"
print(predict(q1,q2))
  

(1, 25)
(1, 768)
(1, 768)
0


# Testing Code

In [51]:
def preprocess_text(x):
    x = str(x).lower()
    x = x.replace(",000,000", "m").replace(",000", "k").replace("′", "'").replace("’", "'")\
                           .replace("won't", "will not").replace("cannot", "can not").replace("can't", "can not")\
                           .replace("n't", " not").replace("what's", "what is").replace("it's", "it is")\
                           .replace("'ve", " have").replace("i'm", "i am").replace("'re", " are")\
                           .replace("he's", "he is").replace("she's", "she is").replace("'s", " own")\
                           .replace("%", " percent ").replace("₹", " rupee ").replace("$", " dollar ")\
                           .replace("€", " euro ").replace("'ll", " will")
    x = re.sub(r"([0-9]+)000000", r"\1m", x)
    x = re.sub(r"([0-9]+)000", r"\1k", x)
    x = re.sub(r"http\S+", "", x)
    x = re.sub('\W', ' ', x)
    
    lemmatizer = WordNetLemmatizer()
    x = lemmatizer.lemmatize(x)
    bfs = BeautifulSoup(x)
    x = bfs.get_text()
    x = x.strip()
    return x

def data_cleaning(data):
    newdata = pd.DataFrame()
    newdata['question1_final'] = data.question1.apply(preprocess_text)
    newdata['question2_final'] = data.question2.apply(preprocess_text)
    return newdata

def doesMatch (q, match):
    q1, q2 = q['question1_final'], q['question2_final']
    q1 = q1.split()
    q2 = q2.split()
    if len(q1)>0 and len(q2)>0 and q1[match]==q2[match]:
        return 1
    else:
        return 0
def feature_extract(data):
    data['q1_char_num'] = data.question1_final.apply(len)
    data['q2_char_num'] = data.question2_final.apply(len)
    data['q1_word_num'] = data.question1_final.apply(lambda x: len(x.split()))
    data['q2_word_num'] = data.question2_final.apply(lambda x: len(x.split()))
    
    data['total_word_num'] = data['q1_word_num'] + data['q2_word_num']
    data['differ_word_num'] = abs(data['q1_word_num'] - data['q2_word_num'])
    data['same_first_word'] = data.apply(lambda x: doesMatch(x, 0) ,axis=1)
    data['same_last_word'] = data.apply(lambda x: doesMatch(x, -1) ,axis=1)
    data['total_unique_word_num'] = data.apply(lambda x: len(set(x.question1_final.split()).union(set(x.question2_final.split()))) ,axis=1)
    data['total_unique_word_withoutstopword_num'] = data.apply(lambda x: len(set(x.question1_final.split()).union(set(x.question2_final.split())) - set(stopwords.words('english'))) ,axis=1)
    data['total_unique_word_num_ratio'] = data['total_unique_word_num'] / data['total_word_num']
    
    data['common_word_num'] = data.apply(lambda x: len(set(x.question1_final.split()).intersection(set(x.question2_final.split()))) ,axis=1)
    data['common_word_ratio'] = data['common_word_num'] / data['total_unique_word_num']
    data['common_word_ratio_min'] = data['common_word_num'] / data.apply(lambda x: min(len(set(x.question1_final.split())), len(set(x.question2_final.split()))) ,axis=1) 
    data['common_word_ratio_max'] = data['common_word_num'] / data.apply(lambda x: max(len(set(x.question1_final.split())), len(set(x.question2_final.split()))) ,axis=1) 
    
    data['common_word_withoutstopword_num'] = data.apply(lambda x: len(set(x.question1_final.split()).intersection(set(x.question2_final.split())) - set(stopwords.words('english'))) ,axis=1)
    data['common_word_withoutstopword_ratio'] = data['common_word_withoutstopword_num'] / data['total_unique_word_withoutstopword_num']
    data['common_word_withoutstopword_ratio_min'] = data['common_word_withoutstopword_num'] / data.apply(lambda x: min(len(set(x.question1_final.split()) - set(stopwords.words('english'))), len(set(x.question2_final.split()) - set(stopwords.words('english')))) ,axis=1) 
    data['common_word_withoutstopword_ratio_max'] = data['common_word_withoutstopword_num'] / data.apply(lambda x: max(len(set(x.question1_final.split()) - set(stopwords.words('english'))), len(set(x.question2_final.split()) - set(stopwords.words('english')))) ,axis=1) 
    
    data["fuzz_ratio"] = data.apply(lambda x: fuzz.ratio(x.question1_final, x.question2_final), axis=1)
    data["fuzz_partial_ratio"] = data.apply(lambda x: fuzz.partial_ratio(x.question1_final, x.question2_final), axis=1)
    data["fuzz_token_set_ratio"] = data.apply(lambda x: fuzz.token_set_ratio(x.question1_final, x.question2_final), axis=1)
    data["fuzz_token_sort_ratio"] = data.apply(lambda x: fuzz.token_sort_ratio(x.question1_final, x.question2_final), axis=1)
    data.fillna(0, inplace=True)
    return data

def getBertEmbeddingsTesting(data):
  tempdata = data
  tempembed = modelST.encode(tempdata.values, device='cuda')
  return  tempembed

def predictAns(x1,x2):
  # data cleaning
  data = [[x1,x2]]
  df = pd.DataFrame(data, columns=['question1','question2'])
  newdata = data_cleaning(df)

  # Feature Extraction
  newdata = feature_extract(newdata)

  # sentenceBERT

  q1_vec = getBertEmbeddingsTesting(newdata.question1_final)
  q2_vec = getBertEmbeddingsTesting(newdata.question2_final)
  q1_vec = q1_vec.reshape((768,))
  q2_vec = q2_vec.reshape((768,))
  distances = []
  dists = cosine_euclidean(q1_vec, q2_vec)
  distances.append(dists)

  distances = np.array(distances)
  distances = pd.DataFrame(distances, columns=['cosine_simlarity_bert', 'euclidean_distance_bert'])
  newdata = pd.concat([newdata, pd.DataFrame(distances)], axis=1)


  #pre processing
  
  newdata.drop(columns=['question1_final', 'question2_final'], inplace=True)
  newdata = newdata.to_numpy()
  scaler = MinMaxScaler()
  scaler.fit(newdata)
  
  newdata = scaler.transform(newdata)

  q1_vec = q1_vec.reshape((1,768))
  q2_vec = q2_vec.reshape((1,768))
  
  return np.hstack((newdata, q1_vec, q2_vec))


q1 = "What are advantages of python?"
q2 = "what are benefits of python?"
print(rf_clf_model.predict(predictAns(q1,q2)))  

[0]


In [52]:
q1 = "What are advantages of python?"
q2 = "what are advantages of java?"
predictAns(q1,q2).shape

(1, 1561)

In [53]:
q1 = "What are advantages of python?"
q2 = "what are advantages of java?"
ans = rf_clf_model.predict(predictAns(q1,q2))
print(ans)

[0]


In [56]:
import joblib

joblib.dump(rf_clf_model,open('/content/drive/MyDrive/Model/model.pkl','wb'))
joblib.dump(modelST,open('/content/drive/MyDrive/Model/modelIST.pkl','wb'))